**(2): Repeat the analysis of Lab 10.9.5 on the IMDb data using a similarly structured neural network. We used 16 hidden units at each of two hidden layers. Explore the effect of increasing this to 32 and 64 units per layer, with and without 30% dropout regularization.**

In [ ]:
!pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 8.4 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=0c377581eb67a5072de8b70d5e3f4a10e3ae74d71d5021f6c2b5b5cc1f33da5a
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting un

In [ ]:
!pip uninstall sympy -y
!pip install sympy

Found existing installation: sympy 1.13.1
Uninstalling sympy-1.13.1:
  Successfully uninstalled sympy-1.13.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 37.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.3 which is incompatible.


In [ ]:
!pip install torchinfo
!pip uninstall pytorch-lightning
!pip install pytorch-lightning
!pip install torchvision

Found existing installation: pytorch-lightning 2.4.0
Uninstalling pytorch-lightning-2.4.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/lightning_fabric/*
    /usr/local/lib/python3.10/dist-packages/pytorch_lightning-2.4.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/pytorch_lightning/*
Proceed (Y/n)? Y
  Successfully uninstalled pytorch-lightning-2.4.0
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
Using cached pytorch_lightning-2.4.0-py3-none-any.whl (815 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 34.1 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3


In [ ]:
# Import necessary libraries
import numpy as np, pandas as pd
from matplotlib.pyplot import subplots
from sklearn.linear_model import (LinearRegression, LogisticRegression, Lasso)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

import ISLP
from ISLP import load_data
from ISLP.models import ModelSpec as MS
from sklearn.model_selection import (train_test_split, GridSearchCV)
import tensorflow as tf
import random
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.optimizers import RMSprop

import torch
#from torch.nn import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from torchmetrics import (MeanAbsoluteError, R2Score)
from torchinfo import summary
from torchvision.io import read_image
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger
from torchvision.datasets import MNIST, CIFAR100
from torchvision.models import (resnet50, ResNet50_Weights)
from torchvision.transforms import (Resize, Normalize, CenterCrop, ToTensor)

from ISLP.torch import (SimpleDataModule, SimpleModule, ErrorTracker, rec_num_workers)
from ISLP.torch.imdb import (load_lookup, load_tensor, load_sparse, load_sequential)


**FOLLOWING CODE SECTIONS COPY PASTED FROM LAB 10.9.5 IN THE TEXTBOOK**

In [ ]:
(imdb_seq_train, imdb_seq_test) = load_sequential(root='data/IMDB')
padded_sample = np.asarray(imdb_seq_train.tensors[0][0])
sample_review = padded_sample[padded_sample > 0][:12]
sample_review[:12]

lookup = load_lookup(root='data/IMDB')
' '.join(lookup[i] for i in sample_review)
max_num_workers=10
(imdb_train, imdb_test) = load_tensor(root='data/IMDB')
imdb_dm = SimpleDataModule(imdb_train, imdb_test, validation=2000, num_workers=min(6, max_num_workers), batch_size=512)

class IMDBModel(nn.Module):
    def __init__(self, input_size):
        super(IMDBModel, self).__init__()
        self.dense1 = nn.Linear(input_size, 16)
        self.activation = nn.ReLU()
        self.dense2 = nn.Linear(16, 16)
        self.output = nn.Linear(16, 1)

    # The forward function should be at the same level as __init__
    def forward(self, x):
        val = x
        for _map in [self.dense1, self.activation, self.dense2, self.activation, self.output]:
            val = _map(val)
        return torch.flatten(val)

imdb_model = IMDBModel(imdb_test.tensors[0].size()[1])
summary(imdb_model, input_size=imdb_test.tensors[0].size(), col_names=['input_size', 'output_size', 'num_params'])

imdb_optimizer = RMSprop(imdb_model.parameters(), lr=0.001)
imdb_module = SimpleModule.binary_classification(imdb_model, optimizer=imdb_optimizer)
imdb_logger = CSVLogger('logs', name='IMDB')
imdb_trainer = Trainer(deterministic=True, max_epochs=30, logger=imdb_logger, callbacks=[ErrorTracker()])
imdb_trainer.fit(imdb_module, datamodule=imdb_dm)
test_res = imdb_trainer.test(imdb_module, datamodule=imdb_dm)
test_res

/usr/local/lib/python3.10/dist-packages/ISLP/torch/imdb.py:131: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  S_test) = [torch.load(_get_imdb(f'IMDB_{r}', root))
/usr/local/

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.853600025177002     │
│         test_loss         │    1.1887187957763672     │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 1.1887187957763672, 'test_accuracy': 0.853600025177002}]

In [ ]:
#from torch.nn import nn
# Data Loading from chunk above and extracting training and test set in
# sequential format
max_num_workers = 10
(imdb_seq_train, imdb_seq_test) = load_sequential(root='data/IMDB')
(imdb_train, imdb_test) = load_tensor(root='data/IMDB')

# Define a new class that inherits from the base class for all NN modules in PyTorch (nn.Module)
class IMDBModel(nn.Module):
    # Initialize the model's parameters as specified in the question with dropout rate for regularization
    def __init__(self, input_size, hidden_units = 16, dropout_rate = 0.0):
        super(IMDBModel, self).__init__()

        # Define layers with configurable hidden units and optional dropout
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_units), # A fully connected layer that maps the input features to the hidden units
            nn.ReLU(),  # ReLU activation function for non-linearity
            nn.Dropout(p = dropout_rate),  # Dropout layer to prevent overfitting
            # Above sequence is repeated with another fully connected layer, ReLU and dropout
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
            nn.Dropout(p = dropout_rate),
            # A fully connected layer that maps the hidden units to a single output unit
            nn.Linear(hidden_units, 1))

    # This method defines the forward pass of the model with input tensor x,
    # passes the input through the defined network layers above and flattens the
    # output tensor to a 1D tensor.
    def forward(self, x):
        return torch.flatten(self.network(x))

# Define a function to train and evaluate model to do with and without 30 %
# dropout regularization.
def train_and_evaluate_model(hidden_units, dropout_rate):
    # Prepare DataModule to load data and to preprocess
    imdb_dm = SimpleDataModule(imdb_train, imdb_test, validation = 2000,
                               num_workers = min(6, max_num_workers),
                               batch_size = 512)
    # Create model using IMDB Model created above using number of features in the test dataset as the input size
    imdb_model = IMDBModel(input_size = imdb_test.tensors[0].size()[1],
                           hidden_units = hidden_units,
                           dropout_rate = dropout_rate)
    # Optimize using RMSProp and set learning rate to 0.01
    imdb_optimizer = RMSprop(imdb_model.parameters(), lr = 0.01)
    # Lightning Module for binary classifiaction using PyTorch Lightning
    imdb_module = SimpleModule.binary_classification(imdb_model, optimizer = imdb_optimizer)
    # Logger for saving training logs
    imdb_logger = CSVLogger('logs', name = f'IMDB_units_{hidden_units}_dropout_{dropout_rate}')
    # Trainer handles the training loop to set the epochs
    imdb_trainer = Trainer(deterministic = True, max_epochs = 30,
                           logger = imdb_logger, callbacks = [ErrorTracker()])
    # Train and Test the model using above functions
    imdb_trainer.fit(imdb_module, datamodule = imdb_dm)
    test_res = imdb_trainer.test(imdb_module, datamodule = imdb_dm)

    # Return results of model evaluation
    return test_res

# Experiment configurations categories to specify the number of hidden units and dropout rate as specified in the question
exper_cat = [{'hidden_units': 16, 'dropout_rate': 0.0},   # Baseline
    {'hidden_units': 32, 'dropout_rate': 0.0},   # Increased hidden units, no dropout
    {'hidden_units': 64, 'dropout_rate': 0.0},   # Increased hidden units, no dropout
    {'hidden_units': 16, 'dropout_rate': 0.3},   # Baseline with dropout
    {'hidden_units': 32, 'dropout_rate': 0.3},   # Increased hidden units with 30 % dropout
    {'hidden_units': 64, 'dropout_rate': 0.3}    # Increased hidden units with 30 % dropout
]

# Run experiments over each category above
res = {}
for exp in exper_cat:
    print(f"Running experiment: {exp}")
    res[f"{exp['hidden_units']} units, {exp['dropout_rate']} dropout"] = train_and_evaluate_model(
        exp['hidden_units'],
        exp['dropout_rate']
    )

# Print res (Note: actual print formatting might need adjustment)
for config, result in res.items():
    print(f"\nConfiguration: {config}")
    print(result)

/usr/local/lib/python3.10/dist-packages/ISLP/torch/imdb.py:131: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  S_test) = [torch.load(_get_imdb(f'IMDB_{r}', root))
/usr/local/

Running experiment: {'hidden_units': 16, 'dropout_rate': 0.0}


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │     0.866919994354248     │
│         test_loss         │    0.8414745926856995     │
└───────────────────────────┴───────────────────────────┘


Configuration: 16 units, 0.0 dropout
[{'test_loss': 1.325154423713684, 'test_accuracy': 0.8548799753189087}]

Configuration: 32 units, 0.0 dropout
[{'test_loss': 1.5540151596069336, 'test_accuracy': 0.8522800207138062}]

Configuration: 64 units, 0.0 dropout
[{'test_loss': 1.1865366697311401, 'test_accuracy': 0.8658000230789185}]

Configuration: 16 units, 0.3 dropout
[{'test_loss': 1.3017518520355225, 'test_accuracy': 0.8689600229263306}]

Configuration: 32 units, 0.3 dropout
[{'test_loss': 0.8540134429931641, 'test_accuracy': 0.8685600161552429}]

Configuration: 64 units, 0.3 dropout
[{'test_loss': 0.8414745926856995, 'test_accuracy': 0.866919994354248}]
